In [1]:

import pandas as pd
import numpy as np
import re
import json
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet
from nltk.tokenize import TreebankWordTokenizer

import pandas as pd
import json
import re

def generate_trending_keywords():
    # Optional NLTK downloads
    nltk.download('punkt')
    nltk.download('stopwords')

    # Load data
    df_search = pd.read_csv('/content/drive/MyDrive/HackFest_Dataset/search_trends.csv')
    df_sales = pd.read_csv('/content/drive/MyDrive/HackFest_Dataset/sales_data.csv')
    df_feedback = pd.read_csv('/content/drive/MyDrive/HackFest_Dataset/customer_feedback.csv')
    df_catalog = pd.read_csv('/content/drive/MyDrive/HackFest_Dataset/product_catalog.csv')

    # Convert dates
    df_search['timestamp'] = pd.to_datetime(df_search['timestamp'])
    df_sales['timestamp'] = pd.to_datetime(df_sales['timestamp'])
    df_feedback['date'] = pd.to_datetime(df_feedback['date'])

    # Preprocess search queries
    stop_words = set(stopwords.words('english'))
    tokenizer = TreebankWordTokenizer()

    def clean_and_tokenize(query):
        query = re.sub(r'[^a-zA-Z\s]', '', str(query).lower())
        tokens = tokenizer.tokenize(query)
        return [word for word in tokens if word not in stop_words]

    df_search['tokens'] = df_search['query'].apply(clean_and_tokenize)
    df_exploded = df_search.explode('tokens')
    df_exploded = df_exploded.rename(columns={'tokens': 'keyword'})

    # Aggregate keyword frequency
    df_token_time = df_exploded.groupby(['timestamp', 'keyword'])['frequency'].sum().reset_index()

    # --- SALES DATA ---
    product_category_map = dict(zip(df_catalog['productId'], df_catalog['category']))
    df_sales['domain'] = df_sales['productId'].map(product_category_map)

    df_sales_agg = df_sales.groupby(['timestamp', 'domain'])['quantitySold'].sum().reset_index()
    df_sales_agg.rename(columns={'quantitySold': 'totalSales'}, inplace=True)

    # --- CUSTOMER FEEDBACK ---
    df_feedback['sentiment'] = df_feedback['commentText'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
    df_feedback['domain'] = df_feedback['productId'].map(product_category_map)

    df_sentiment = df_feedback.groupby(['date', 'domain'])['sentiment'].mean().reset_index()
    df_sentiment.rename(columns={'date': 'timestamp', 'sentiment': 'avgSentiment'}, inplace=True)

    # --- Map keywords to domain ---
    catalog_keywords = set()
    for col in ['title', 'category', 'modifiers', 'keywords']:
        text = " ".join(df_catalog[col].dropna().astype(str).str.lower())
        catalog_keywords.update(tokenizer.tokenize(text))
    catalog_keywords = list(catalog_keywords)

    def guess_domain(token):
        for domain in df_catalog['category'].unique():
            if token in domain.lower():
                return domain
        return 'unknown'

    df_token_time['domain'] = df_token_time['keyword'].apply(guess_domain)

    # Merge with sales and sentiment
    df_all = df_token_time.merge(df_sales_agg, on=['timestamp', 'domain'], how='left')
    df_all = df_all.merge(df_sentiment, on=['timestamp', 'domain'], how='left')
    df_all['totalSales'] = df_all['totalSales'].fillna(0)
    df_all['avgSentiment'] = df_all['avgSentiment'].fillna(0)

    # Normalize features
    scaler = MinMaxScaler()
    df_all[['frequency_norm', 'sales_norm', 'sentiment_norm']] = scaler.fit_transform(
        df_all[['frequency', 'totalSales', 'avgSentiment']]
    )

    # Compute composite score
    w_search, w_sales, w_sentiment = 0.5, 0.3, 0.2
    df_all['composite_score'] = (
        w_search * df_all['frequency_norm'] +
        w_sales * df_all['sales_norm'] +
        w_sentiment * df_all['sentiment_norm']
    )

    # --- Forecasting ---
    forecast_dict = {}
    top_keywords = df_all.groupby('keyword')['composite_score'].sum().sort_values(ascending=False).head(50).index.tolist()

    for keyword in top_keywords:
        sub_df = df_all[df_all['keyword'] == keyword][['timestamp', 'composite_score']]
        sub_df = sub_df.rename(columns={'timestamp': 'ds', 'composite_score': 'y'})

        if len(sub_df) >= 5:
            model = Prophet(daily_seasonality=True)
            model.fit(sub_df)

            future = model.make_future_dataframe(periods=7)
            forecast = model.predict(future)

            total_future_score = forecast.tail(7)['yhat'].sum()
            forecast_dict[keyword] = total_future_score

    # Sort and save top 20 trending keywords
    sorted_keywords = sorted(forecast_dict.items(), key=lambda x: x[1], reverse=True)
    top_keywords_output = [{'keyword': kw, 'forecast_score': round(score, 2)} for kw, score in sorted_keywords[:20]]

    with open('/content/drive/MyDrive/HackFest_Dataset/toptrendingkeywords.json', 'w') as f:
        json.dump(top_keywords_output, f, indent=4)

    print("✅ Saved top trending keywords to toptrendingkeywords.json")

# You can call this function to run
# generate_trending_keywords()



def generate_recommendations():
    # 1. Load trending keywords
    with open('/content/drive/MyDrive/HackFest_Dataset/toptrendingkeywords.json', 'r') as f:
        trending_keywords = json.load(f)

    # Convert to DataFrame
    df_trending = pd.DataFrame(trending_keywords)

    # 2. Load product catalog
    df_catalog = pd.read_csv('/content/drive/MyDrive/HackFest_Dataset/product_catalog.csv')

    # Fill NaNs with empty strings to avoid issues
    df_catalog.fillna('', inplace=True)

    # 3. Combine product details for matching
    df_catalog['combined_text'] = (
        df_catalog['title'].astype(str) + ' ' +
        df_catalog['category'].astype(str) + ' ' +
        df_catalog['modifiers'].astype(str) + ' ' +
        df_catalog['keywords'].astype(str)
    ).str.lower()

    # 4. Match products to trending keywords
    matched_products = []

    for _, row in df_trending.iterrows():
        keyword = row['keyword'].lower()
        score = row['forecast_score']

        # Find products containing the keyword
        matches = df_catalog[df_catalog['combined_text'].str.contains(rf'\b{re.escape(keyword)}\b', regex=True)].copy()
        matches['matched_keyword'] = keyword
        matches['forecast_score'] = score

        matched_products.append(matches)

    # 5. Concatenate all matched products
    if matched_products:
        df_matched_products = pd.concat(matched_products, ignore_index=True)

        # Optional: sort by forecast score
        df_matched_products = df_matched_products.sort_values(by='forecast_score', ascending=False)

        # 6. Save or display the top results
        df_matched_products[['productId', 'title', 'category', 'matched_keyword', 'forecast_score']].to_csv('/content/drive/MyDrive/HackFest_Dataset/recommended_products.csv', index=False)

        print("Recommended products saved to recommended_products.csv")
    else:
        print("No products matched the trending keywords.")
  # generate_trend()



In [2]:
generate_trending_keywords()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/HackFest_Dataset/search_trends.csv'

In [ ]:
generate_recommendations()

Recommended products saved to recommended_products.csv


In [ ]:
!pip install dash plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [ ]:
import dash
from dash import html, dcc, dash_table
import plotly.express as px
import pandas as pd
import json

# Import your functions
#import generate_trending_keywords, generate_recommendations

def main():
    # --- 1. Generate Trending Keywords and Recommendations ---
    print("🔍 Generating trending keywords and recommendations...")
    #generate_trending_keywords()
    generate_recommendations()
    print("✅ Data generation complete!")

    # --- 2. Load Processed Data ---
    with open('/content/drive/MyDrive/HackFest_Dataset/toptrendingkeywords.json', 'r') as f:
        trending_keywords = json.load(f)
    df_trending = pd.DataFrame(trending_keywords)

    df_recommendations = pd.read_csv('/content/drive/MyDrive/HackFest_Dataset/recommended_products.csv')

    # --- 3. Create Dash App ---
    app = dash.Dash(__name__)
    app.title = "AI-Powered Trend & Recommendation Dashboard"

    # --- 4. Visualizations ---
    # Top Trending Keywords Bar Chart
    trending_fig = px.bar(
        df_trending,
        x='keyword',
        y='forecast_score',
        title="🔥 Top Trending Keywords Forecast",
        color='forecast_score'
    )

    # Recommended Products Table
    recommendation_table = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in df_recommendations.columns],
        data=df_recommendations.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'}
    )

    # --- 5. Layout with Tabs ---
    app.layout = html.Div([
        html.H1("🧠 AI-Powered Trend & Recommendation Dashboard", style={'textAlign': 'center'}),
        dcc.Tabs([
            dcc.Tab(label='Trending Keywords', children=[
                html.H2("Top 20 Trending Keywords"),
                dcc.Graph(figure=trending_fig),
                html.H4("🔍 Data Preview"),
                dash_table.DataTable(
                    columns=[{"name": i, "id": i} for i in df_trending.columns],
                    data=df_trending.to_dict('records'),
                    page_size=10,
                    style_table={'overflowX': 'auto'},
                    style_cell={'textAlign': 'left'}
                )
            ]),
            dcc.Tab(label='Recommended Products', children=[
                html.H2("📦 Recommended Products"),
                recommendation_table
            ])
        ])
    ])

    # --- 6. Run App ---
    app.run(debug=True)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'dash'